# Bayesian Personalized Ranking
[BPR: Bayesian Personalized Ranking from Implicit Feedback](https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf)

The paper puts forth something they call “BRP-Opt”, a generic optimization criterion for optimal personalized ranking. Basically what this means is an approach that can be applied to different types of recommendation models like Matrix Factorization or k-Nearest-Neighbour (that's the “generic” part) and that solves a ranking for a set of items for a given user.

Whereas most other optimizers just look at if a user interacted with an item, BPR looks at the user, one item the user interacted with and one item the user did not (the unknown item). This gives us a triplet **(u, i, j)** of a user **(u)**, one known item **(i)** and one unknown item **(j)**.

We can express the relationship between known and unknown items like this:
$$\hat{x}_ui - \hat{x}_uj > 0 $$

Here $\hat{x}$<sub>ui</sub> denotes the score for user 'u' and a known item 'i' and $\hat{x}$<sub>uj</sub> the score for user 'u' and unknown item 'j'. The above condition is satisfied if the score for the known item is larger than that of the unknown one.

### Bayesian Formulation

Posterior probability is proportional to the likelihood multiplied by the prior probability:
<i>Posterior probability ∝ Likelihood x Prior probability</i>

So what we want to do is to use a formulation to update the probability of our hypothesis as more events/information becomes available, i.e we want to maximize the probability of it being true.

### A Closer Look At The Math

Using the formula <i>Posterior probability ∝ Likelihood x Prior probability</i> the posterior probability we want to maximize in this case is:

$$p(\theta | >_u) ∝ p(>_u | \theta) x p(\theta)$$
Here $>_u$ is a latent preference structure for user **u** and $\theta$ represents the parameters of some kind of model, like matrix factorization or kNN.

So basically we want to maximize the probability of parameters $\theta$ given a latent preference structure for user $>_u$. The paper shows us that the product of the likelihood $p(>_u | \theta)$ is equal to the product of $p(i <_u j | \theta)$:
<img src="form1.png"  style="width: 400px">
Next, we define the likelihood that a given user actually prefers the known item i over unknown item j as the following:
<img src="form2.png" style="width: 300px">
Here $\sigma$ is the sigmoid function and $\hat{x}_{uij}(\theta)$ is some kind of function that models the relationship between a user, a known item and an unknown item given a set of parameters. BPR does not dictate what function this is and can, therefore, be used with a number of different model classes. In our case this function is the difference between the score of **u and j** subtracted from the score of **u and i**:
<img src="form3.png" style="width: 200px">
We can then rewrite it to use the sigmoid function for the likelihood. The paper also suggests using ln sigmoid (natural log) based on their MLE (Maximum Likelihood Evaluation):
<img src="form4.png" style="width: 300px">
Remembering the logarithm product rule ( ln(a * b) = ln(a) + ln(b) ) we can now change the whole equation to:
<img src="form5.png" style="width: 350px">
Based on this we then arrive at the final optimization criterion for our model:
<img src="form6.png" style="width: 300px">
* $\theta$ : Our model parameter vector.
* $\hat{x}_{uij}$ : Relationship between (u, i, j). Here the score of (u, j) subtracted from the score of (u, j).
* **ln** : The natural logarithm.
* $\sigma$ : The logistic sigmoid.
* $\lambda$ : LAmbda, the regularization hyperparameter for our model.
* $||\theta||$ : The L2 norm of our model parameters.

### The Tensorflow Model

Follolwing are some of the nodes we will be using when building our graph:
* **Variables**: Nodes that maintain their state in the graph across multiple executions. These are the things in our graph we want to train in order to minimize the loss function.
* **Embedding lookup**: Lets us retrieve a row from a tensor given its index. Similar to indexing into a numpy array.
* **Optimizer**: There are different types of optimizers but what they do is to calculate the loss function and then update the values in the graph for the next run in order to minimize that loss.

### The Model

We will implement BPR using matrix factorization which means taking the interaction matrix <b>R</b> of size (all users x all items) and factoring it into matrix <b>U</b> of size (all users x latent features) and <b>V</b> of size (latent features x all items). So when we’re done want <b>R ≈ U x V</b>.
<img src="matrix_fact.png" style="width: 800px">
We will use Stochastic Gradient Descent (SGD) to arrive at this approximation. It works by initializing <b>U</b> and <b>V</b> with uniformly random values and then, using the optimization criterion from before, continuously updating them with new values to maximize the posterior probability i.e. getting a higher and higher probability for <b>R = U x V</b>.

Let’s have a look at a simplified illustration of the graph we will be implementing and how data will flow through it.
<img src="model.png" style="width: 800px">
So we start at the top with our inputs (placeholders) which will be a user id <b>u</b>, a known item id <b>i</b> and unknown item id <b>j</b>. We then also randomly initialize our matrixes <b>U</b> and <b>V</b> (variables).

From here we create three embeddings, one for the user factors, known item factors and unknown item factors. We take these embeddings and multiply them along with a bias to get $\hat{x}_{ui}$ and $\hat{x}_{uj}$ respectively.

Next, we take $\hat{x}_{ui}$ and $\hat{x}_{uj}$, compute $\hat{x}_{uij}$ and feed it into the negative natural log — of the sigmoid — of $\hat{x}_{uij}$. So we get $-ln \sigma(\hat{x}_{uij})$  

Finally, we add the <b>L2</b> norm to $-ln \sigma(\hat{x}_{uij})) and arrive at the final loss function:
<img src="form7.png" style="width: 200px">

Let’s expand this out to see it in the form it will actually be implemented in code:
<img src="form8.png" style="width: 400px">

## Dataset

As before we’ll be using the lastfm dataset containing the listening behavior of 360,000 users. It contains the user id, an artist id, the name of the artists and the number of times a user played any given artist.

## Implementation

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm

##### Processing Data

In [3]:
df = pd.read_csv('usersha1-artmbid-artname-plays.tsv', sep='\t')

In [5]:
df.columns = ['user','artistid','artist','plays']

In [8]:
df.head()

,user,artist,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691


In [7]:
df.drop(['artistid'],axis=1,inplace=True)

In [10]:
#Dropping columns with missing values
df = df.dropna()

In [11]:
# Convert artists names into numerical IDs
df['user_id'] = df['user'].astype("category").cat.codes
df['artist_id'] = df['artist'].astype("category").cat.codes

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
item_lookup = df[['artist_id', 'artist']].drop_duplicates()
item_lookup['artist_id'] = item_lookup.artist_id.astype(str)

In [13]:
df = df.drop(['user', 'artist'], axis=1)

In [14]:
# Drop any rows with 0 plays
df = df.loc[df.plays != 0]

In [15]:
# Create lists of all users, artists and plays
users = list(np.sort(df.user_id.unique()))
artists = list(np.sort(df.artist_id.unique()))
plays = list(df.plays)

In [16]:
# Get the rows and columns for our new matrix
rows = df.user_id.astype(float)
cols = df.artist_id.astype(float)

# Contruct a sparse matrix for our users and items containing number of plays
data_sparse = sp.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

In [17]:
uids, iids = data_sparse.nonzero()

##### Hyperparameters

In [18]:
epochs = 50
batches = 30
num_factors = 64 # Number of latent features

In [19]:
# Independent lambda regularization values 
# for user, items and bias.
lambda_user = 0.0000001
lambda_item = 0.0000001
lambda_bias = 0.0000001

In [20]:
# Our learning rate 
lr = 0.005

In [21]:
# How many (u,i,j) triplets we sample for each batch
samples = 15000

##### Tensorflow Graph

In [22]:
graph = tf.Graph()

We create one function that just initializes a Tensorflow variable with random values, one that uses that function to create a variable and then embeds it using an embedding lookup and one for getting the value of a variable back given its name.

In [23]:
def init_variable(size, dim, name=None):
    # Helper function to initialize a new variable with uniform random values.
    std = np.sqrt(2 / dim)
    return tf.Variable(tf.random_uniform([size, dim], -std, std), name=name)


In [24]:
def embed(inputs, size, dim, name=None):
    # Helper function to get a Tensorflow variable and create an embedding lookup to map our user and itemindices to vectors.
    emb = init_variable(size, dim, name)
    return tf.nn.embedding_lookup(emb, inputs)

In [25]:
def get_variable(graph, session, name):
    # Helper function to get the value of a Tensorflow variable by name.
    v = graph.get_operation_by_name(name)
    v = v.values()[0]
    v = v.eval(session=session)
    return v

First, we create placeholders for u, i and j that we can later feed data into. We initialize our *U* and *V* matrixes and create embeddings for *u_factors, i_factors*, and *j_factors*. We also create embeddings for our biases *i_bias* and *j_bias*.

Next, we calculate the score for our user and item *i* as well as for item *j* to get x̂ui and x̂uj and then we compute *x̂uij*.

Here we also the AUC score and add it to our Tensorflow summary so we can monitor it during training.

For the L2 norm, all we need to do is to square each parameter multiplied with the regularization value and then add them all together. Taking the negative natural log of the sigmoid of *x̂uij*, plus the L2 norm gives us our final loss function. Lastly, we use the built-in Adam optimizer to minimize our loss.

In [35]:
with graph.as_default():
    
    #Loss function: 
    #-SUM ln σ(xui - xuj) + λ(w1)**2 + λ(w2)**2 + λ(w3)**2 ...
    #ln = the natural log
    #σ(xuij) = the sigmoid function of xuij.
    #λ = lambda regularization value.
    #||W||**2 = the squared L2 norm of our model parameters.
    
    # Input into our model, in this case our user (u),
    # known item (i) an unknown item (i) triplets.
    u = tf.placeholder(tf.int32, shape=(None, 1))
    i = tf.placeholder(tf.int32, shape=(None, 1))
    j = tf.placeholder(tf.int32, shape=(None, 1))
    
    # User feature embedding
    u_factors = embed(u, len(users), num_factors, 'user_factors') # U matrix

    # Known and unknown item embeddings
    item_factors = init_variable(len(artists), num_factors, "item_factors") # V matrix
    i_factors = tf.nn.embedding_lookup(item_factors, i)
    j_factors = tf.nn.embedding_lookup(item_factors, j)

    # i and j bias embeddings.
    item_bias = init_variable(len(artists), 1, "item_bias")
    i_bias = tf.nn.embedding_lookup(item_bias, i)
    i_bias = tf.reshape(i_bias, [-1, 1])
    j_bias = tf.nn.embedding_lookup(item_bias, j)
    j_bias = tf.reshape(j_bias, [-1, 1])

    # Calculate the dot product + bias for known and unknown
    # item to get xui and xuj.
    xui = i_bias + tf.reduce_sum(u_factors * i_factors, axis=2)
    xuj = j_bias + tf.reduce_sum(u_factors * j_factors, axis=2)
    
    # We calculate xuij.
    xuij = xui - xuj

    # Calculate the mean AUC (area under curve).
    # if xuij is greater than 0, that means that 
    # xui is greater than xuj (and thats what we want).
    #u_auc = tf.reduce_mean(tf.to_float(xuij > 0))
    if xuij>0:
        u_auc = tf.reduce_mean(tf.cast(xuij,tf.float32)
    
    # Output the AUC value to tensorboard for monitoring.
    tf.summary.scalar(u_auc)

    # Calculate the squared L2 norm ||W||**2 multiplied by λ.
    l2_norm = tf.add_n([
        lambda_user * tf.reduce_sum(tf.multiply(u_factors, u_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(i_factors, i_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(j_factors, j_factors)),
        lambda_bias * tf.reduce_sum(tf.multiply(i_bias, i_bias)),
        lambda_bias * tf.reduce_sum(tf.multiply(j_bias, j_bias))
        ])
    # Calculate the loss as ||W||**2 - ln σ(Xuij)
    #loss = l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(xuij)))
    loss = -tf.reduce_mean(tf.log(tf.sigmoid(xuij))) + l2_norm
    
    # Train using the Adam optimizer to minimize 
    # our loss function.
    opt = tf.train.AdamOptimizer(learning_rate=lr)
    step = opt.minimize(loss)

    # Initialize all tensorflow variables.
    init = tf.global_variables_initializer()

SyntaxError: invalid syntax (<ipython-input-35-f2220bbc0801>, line 47)

For each epoch and batch we defined in our hyperparameters we sample a number of random indices from our dataset and then get user ids (u) and known item ids (i) matching those indices. We then sample 15 000 random unknown items (j).

These items go into our feed_dict which is a dictionary we will feed into our graph. We then run the graph with those values, update the progress bar and display the current loss and AUC values.

In [31]:
# Run the session. 
session = tf.Session(config=None, graph=graph)
session.run(init)

# This has noting to do with tensorflow but gives
# us a nice progress bar for the training.
progress = tqdm(total=batches*epochs)

for _ in range(epochs):
    for _ in range(batches):

        # We want to sample one known and one unknown 
        # item for each user. 

        # First we sample 15000 uniform indices.
        idx = np.random.randint(low=0, high=len(uids), size=samples)

        # We then grab the users matching those indices.
        batch_u = uids[idx].reshape(-1, 1)

        # Then the known items for those users.
        batch_i = iids[idx].reshape(-1, 1)
        
        # Lastly we randomly sample one unknown item for each user.
        batch_j = np.random.randint(low=0, high=len(artists), size=(samples, 1), dtype='int32')

        # Feed our users, known and unknown items to
        # our tensorflow graph. 
        feed_dict = { u: batch_u, i: batch_i, j: batch_j }

        # We run the session.
        var, l, auc = session.run([step, loss, u_auc], feed_dict)
    
    progress.update(batches)
    progress.set_description('Loss: %.3f | AUC: %.3f' % (l, auc))

progress.close()
    





  0%|          | 0/1500 [00:00<?, ?it/s]

InternalError: Unsupported feed type